<body>
    <h1>Course Project</h1>
    <p>Scraping a local job site for data scientist jobs in HK</p>
</body>

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

import re
import pandas as pd
import math

In [2]:
def get_job_text(job_link):
    _page = requests.get(job_link)
    _soup = BeautifulSoup(_page.text, "html.parser")
    #print(job_link)
    x = _soup.find(name="div", attrs={"class":"jobsearch-JobComponent-description"})
    if x:
        return x.text
    else:
        return "Not Found"

In [4]:
#URL = "https://www.indeed.hk/viewjob?jk=2dfb473af85953b5"
#URL = "https://www.indeed.hk/viewjob?jk=6567706422d33abe"
#get_job_text(URL)

In [11]:
start = 0
end = 100

columns = ["job_title", "company_name", "details_link", "job_text"]
jobs_df = pd.DataFrame(columns = columns)


while True:
    url = "https://www.indeed.hk/jobs?q=%22data+scientist%22&start=" + str(start)
    time.sleep(1)  # ensuring at least 1 second between page grabs
    
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    
    # how many jobs there are?
    if start == 0:
        # a loop here is silly I know
        for div in soup.find_all(name="meta" , attrs={"name":"description"}):
            matchObj = re.match( r"^([0-9]+)", div['content'])
            num_jobs = matchObj.group()
            end = math.floor((int(num_jobs)-1)/10) * 10
            # data frame will contain more records, maybe because of duplicated sponsored jobs
            # and also our last page contains more jobs than it should.
            print("There are ", num_jobs, "jobs.")
            print("start index for sites:", start, "ending index:", end)
            #end = 20

    print(url)
    start = start + 10
    # somehow the main loop, for each single job per page (should be 16 jobs, but seems not always)
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        num = (len(jobs_df) + 1)
        job_post = []
        
        # job title
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            job_post.append(a["title"])
        
        # job company
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                job_post.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                job_post.append(span.text.strip())

        job_link = "https://www.indeed.hk/viewjob?jk=" + div["data-jk"]
        job_post.append(job_link)
        job_text = get_job_text(job_link)
        
        #job_post.append(job_text[0:10])
        job_post.append(job_text)
        
        #print(job_post)
        jobs_df.loc[num] = job_post
        
    if start > end:
        break
    

There are  79 jobs.
start index for sites: 0 ending index: 70
https://www.indeed.hk/jobs?q=%22data+scientist%22&start=0
https://www.indeed.hk/jobs?q=%22data+scientist%22&start=10
https://www.indeed.hk/jobs?q=%22data+scientist%22&start=20
https://www.indeed.hk/jobs?q=%22data+scientist%22&start=30
https://www.indeed.hk/jobs?q=%22data+scientist%22&start=40
https://www.indeed.hk/jobs?q=%22data+scientist%22&start=50
https://www.indeed.hk/jobs?q=%22data+scientist%22&start=60
https://www.indeed.hk/jobs?q=%22data+scientist%22&start=70


Save this now to a file. Further processing will be done in another script.

In [13]:
jobs_df.to_csv("ds_jobs_indeed_v01_20181101.csv", index=False)

In [14]:
jobs_df.head()
#print(jobs_df.iloc[0,3])
# need to check for duplcates, there are a lot.
sum(jobs_df.duplicated())


48

In [15]:
jobs_df.shape #(128, 4)


(128, 4)

END OF SCRIPT

The rest is templates.

In [99]:
#URL = "https://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"
#URL = "https://www.indeed.hk/jobs?q=%22data+scientist%22&l="
URL = "https://www.indeed.hk/jobs?q=%22data+scientist%22"
URL = "https://www.indeed.hk/viewjob?jk=2dfb473af85953b5"
URL = "https://www.indeed.hk/viewjob?jk=6567706422d33abe"



#    https://www.indeed.hk/jobs?q=%22data+scientist%22&l=
#conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, "html.parser")
#printing soup in a more structured tree format that makes for easier reading
#print(soup.prettify())

#for x in soup.find_all(name="div", attrs={"class":"jobsearch-JobComponent-description"}):
#    print(x.text)
#with open("indeed_html.txt", "w") as text_file:
#    text_file.write(soup.prettify())
x = soup.find(name="div", attrs={"class":"jobsearch-JobComponent-description"})
#print(x.text)




In [70]:
def extract_job_title_from_result(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        #print(div)
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            print(a)
            jobs.append(a["title"])
    return(jobs)
extract_job_title_from_result(soup)

<a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0C1KiKFNLSTeoagfKR_iBMdGU9SgqzP7Goclj1knC2rZz8RByLEz8BsgzjaLoQa0qioYCsrfsEiWnLOOivI-1CXyZ5ODDVkudJreSEG7GCpk_65j_UwhX55pdMnIfYmIbURZ9FMTtCdSjmLBdfn_PR-hK3fgyoD5pGFlu32lqxNK08UTHGSVYzVzpRgic1AkwcxgBTVruRB8v17rhC1qiV40EcriOabpzQe5Ld8SpQKQyxGQyq6B1Q3hNUzTdIkniHL0Y5kaIj8npodWAf39v4UbB8S9hmhyDlOkfMPzR40HEI4AwNOWeOdhbleUheBuyqv4NEdjaSljhDZ62Ki1MRXUufTPXdByccVVlCQnyZJZ9O0s8eXTbuQIYWdImXCKMygLtp5pM96pdOPBwePE7pl2uSbK-Ky2A9MDm7d-ii0Uc1FbSjB9bpwnejreH5HQ5HKp9kt2RRjEVs6S7My_vwppyZTByqVIaqlPUrgmp04FBc3u2S_1fH2qf4amUfh-nPgQHHddjgDbhGCo5D4k9OTcUXqKwHUAoCQhdLpAIOGPbLHsaoctz2oLz7U9y64N-osxnmzlrA-Iwwn4NUwTNrhiBYEIknbXIII9y2cgHkkr7GiLOSs_yrvaJbm9JEGsqU6mTbU9VaNnvgXCbFBcMGivXgww_m1Z4Uiicp4S3dwc6kJvGmYX4DQ7wda_rJM5jA=&amp;vjs=3&amp;p=1&amp;sk=&amp;fvj=0" id="sja1" onclick="setRefineByCookie([]); sjoc('sja1',0); convCtr('SJ')" onmousedown="sjomd('sja1'); clk('sja1');" rel="noopener nofollow" target="_blank" 

['Data Scientist',
 'Data Scientist',
 'Data Scientist',
 'Supply Chain Data Scientist',
 'Data Scientist',
 'Data Scientist',
 'Data Scientist',
 'Data Scientist / Quantitative Analyst',
 'Data Scientist - Hong Kong',
 'Data Scientist / Computational Scientist',
 'Research Data Scientist /Big Data Engineer,AI Dept',
 'Junior Data Scientist',
 'Data Scientist',
 'Data Scientist - Media',
 'Data Scientist',
 'Data Scientist / Computational Scientist']

In [32]:
# https://www.indeed.hk/viewjob?jk=7400947080a2c1ef

def extract_link_for_details(soup):
    link_details = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
#        print(div["data-jk"])
        link_details.append("https://www.indeed.hk/viewjob?jk=" + div["data-jk"])
    return(link_details)

extract_link_for_details(soup)

['https://www.indeed.hk/viewjob?jk=8fac5bf3bd04d950',
 'https://www.indeed.hk/viewjob?jk=1ab9ad946450576f',
 'https://www.indeed.hk/viewjob?jk=f1621540076157c1',
 'https://www.indeed.hk/viewjob?jk=6567706422d33abe',
 'https://www.indeed.hk/viewjob?jk=d92b5c3ca5f3d1f7',
 'https://www.indeed.hk/viewjob?jk=2dfb473af85953b5',
 'https://www.indeed.hk/viewjob?jk=b3b7b9e98a61c1fa',
 'https://www.indeed.hk/viewjob?jk=8065aee734f99b7c',
 'https://www.indeed.hk/viewjob?jk=dec891350591d503',
 'https://www.indeed.hk/viewjob?jk=b4ecafcc9ba3d85b',
 'https://www.indeed.hk/viewjob?jk=608c233395777324',
 'https://www.indeed.hk/viewjob?jk=a08880d50ab5b9b2',
 'https://www.indeed.hk/viewjob?jk=65b3be9b13fea89b',
 'https://www.indeed.hk/viewjob?jk=c1c83d3135356139',
 'https://www.indeed.hk/viewjob?jk=a59b9777a43f6a69',
 'https://www.indeed.hk/viewjob?jk=3d6b604aed24017e']

In [34]:
# the details page

URL = "https://www.indeed.hk/viewjob?jk=8fac5bf3bd04d950"
page_details = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup_details = BeautifulSoup(page_details.text, "html.parser")
#printing soup in a more structured tree format that makes for easier reading
#print(soup_details.prettify())

with open("indeed_details_html.txt", "w") as text_file:
    text_file.write(soup_details.prettify())

In [18]:
def extract_company_from_result(soup):
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)
 
extract_company_from_result(soup)

['Terminal 1',
 'Cogs Agency HK',
 'RADICA SYSTEMS LIMITED',
 'BAH Partners',
 'Lab Viso Limited',
 'Cathay Pacific',
 'AXA',
 'Neo Derm Ltd.',
 'IBM',
 'Thinkcol.AI',
 'AUREXIA Consulting',
 'Libbler',
 'Credit Suisse',
 'SOUTH CHINA MORNING POST PUBLISHERS LTD',
 'Hotmob Limited',
 'ELEVATE HONG KONG HOLDINGS LIMITED']

In [20]:
def extract_summary_from_result(soup):
    summaries = []
    spans = soup.findAll('span', attrs={'class': 'summary'})
    for span in spans:
        summaries.append(span.text.strip())
    return(summaries)

extract_summary_from_result(soup)

['Data Visualization skills. Design data collection, integration and retention requirements. Conduct data analysis and create algorithms to predict probabilities....',
 'They are looking to hire a Data Scientist, Director, to help enhancing such capabilities. Solid experience in data science, machine learning, Natural Language...',
 'Implement data science and data analytic models in common languages. Devise technical data analytic or data science solutions to open business problem....',
 'Experience with Big Data. 2-8 years years of experience working as a quantitative analyst or data scientist in any area....',
 'We are looking for a talented Data Scientist to join our team. Analyse unstructured data (e.g. (i) extract entity from the unstructured data;...',
 'Essential to have strong understanding in database concepts & data modelling, business intelligence disciplines & data management tools....',
 'Data Scientist (180003VK). Big data/ Data Modelling. 3+ years of relevant working ex

In [19]:
def extract_salary_from_result(soup):
    salaries = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            salaries.append(div.find('nobr').text)
        except:
            try:
                div_two = div.find(name="div", attrs={"class":"sjcl"})
                div_three = div_two.find("div")
                salaries.append(div_three.text.strip())
            except:
                salaries.append("Nothing_found")
    return(salaries)
extract_salary_from_result(soup)

['Terminal 1',
 'Cogs Agency HK',
 'RADICA SYSTEMS LIMITED',
 'BAH Partners',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Hotmob Limited',
 'ELEVATE HONG KONG HOLDINGS LIMITED']